In [1]:
import torch

In [2]:
from datasets import load_dataset

fever_plus = load_dataset("tommasobonomo/sem_augmented_fever_nli")

c:\Users\damic\anaconda3\envs\nlp_env2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd
pd.DataFrame(fever_plus["train"][0])

,id,premise,hypothesis,label,wsd,srl
premise,150448,Roman Atwood . He is best known for his vlogs ...,Roman Atwood is a content creator.,ENTAILMENT,"[{'index': 0, 'text': 'Roman', 'pos': 'ADJ', '...","{'tokens': [{'index': 0, 'rawText': 'Roman'}, ..."
hypothesis,150448,Roman Atwood . He is best known for his vlogs ...,Roman Atwood is a content creator.,ENTAILMENT,"[{'index': 0, 'text': 'Roman', 'pos': 'PROPN',...","{'tokens': [{'index': 0, 'rawText': 'Roman'}, ..."


In [4]:
# Load model directly
from transformers import AutoModel
model = AutoModel.from_pretrained("microsoft/deberta-v3-base")

print(f"auto_model class is : {type(model)}")

c:\Users\damic\anaconda3\envs\nlp_env2\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


auto_model class is : <class 'transformers.models.deberta_v2.modeling_deberta_v2.DebertaV2Model'>


In [5]:
from transformers import AutoTokenizer, Trainer, TrainingArguments

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

sequence = "Using a Transformer network is simple; Transform it ; form it"
tokens = tokenizer.tokenize(sequence)
#decoded_string = tokenizer.decode([7993, 170, 11303, 1200, 2443, 1110, 3014])


c:\Users\damic\anaconda3\envs\nlp_env2\lib\site-packages\transformers\convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [17]:
tokens = tokenizer.tokenize(fever_plus["test"]["premise"][100])
len(tokens)

19

In [8]:
# Load the pre-trained DeBERTa model and tokenizer
model = AutoModel.from_pretrained("microsoft/deberta-v3-base", num_labels=3)
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")


# Tokenize the dataset
train_input = tokenizer(fever_plus["train"]["premise"], fever_plus["train"]["hypothesis"], padding='max_length', truncation=True, max_length=128, return_tensors="pt")
val_input = tokenizer(fever_plus["validation"]["premise"], fever_plus["validation"]["hypothesis"], padding='max_length', truncation=True, max_length=128, return_tensors="pt")
test_input = tokenizer(fever_plus["test"]["premise"], fever_plus["test"]["hypothesis"], padding='max_length', truncation=True, max_length=128, return_tensors="pt")


c:\Users\damic\anaconda3\envs\nlp_env2\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
c:\Users\damic\anaconda3\envs\nlp_env2\lib\site-packages\transformers\convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [9]:
train_input[0]

Encoding(num_tokens=128, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [10]:
# free up CUDA space
torch.cuda.empty_cache()

In [ ]:
# # Rename label column
# train_input = train_input.rename_column("label", "labels")
# val_input = val_input.rename_column("label", "labels")
# test_input = test_input.rename_column("label", "labels")


# # Set the format of the dataset
# train_input.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
# val_input.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
# test_input.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# # keys expected by the model: input_ids,attention_mask,token_type_ids,position_ids,inputs_embeds,output_attentions,output_hidden_states,return_dict,label_ids,label

In [ ]:
# # Prepare the data for training
# train_input = train_input.remove_columns(["text"])
# train_input.set_format("torch")


# train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10000))  # Select a smaller subset for quick training
# eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))  # Select a smaller subset for quick evaluation

In [ ]:
'''
# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Create the Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_input,
    eval_dataset=val_input,
)

# Fine-tune the model
trainer.train()

# Evaluate the model
trainer.evaluate()
'''

In [ ]:
# from transformers import DebertaTokenizer, DebertaForSequenceClassification, Trainer, TrainingArguments
# from datasets import load_dataset
# import torch

# # Load the dataset
# dataset = load_dataset("imdb")

# # Load the pre-trained DeBERTa model and tokenizer
# model_name = "microsoft/deberta-base"
# tokenizer = DebertaTokenizer.from_pretrained(model_name)
# model = DebertaForSequenceClassification.from_pretrained(model_name, num_labels=2)

# # Tokenize the dataset
# def tokenize_function(examples):
#     return tokenizer(examples["text"], padding="max_length", max_length=128, truncation=True)

# tokenized_datasets = dataset.map(tokenize_function, batched=True)

# # Prepare the data for training
# tokenized_datasets = tokenized_datasets.remove_columns(["text"])
# tokenized_datasets.set_format("torch")

# train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10000))  # Select a smaller subset for quick training
# eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))  # Select a smaller subset for quick evaluation

# # Define the training arguments
# training_args = TrainingArguments(
#     output_dir="./results",
#     eval_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=1,
#     weight_decay=0.01,
# )

# # Create the Trainer instance
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=eval_dataset,
# )

# # Fine-tune the model
# trainer.train()

# import pickle
# # Save the trained model to a file
# with open('model.pkl', 'wb') as file:
#     pickle.dump(model, file)

# # Evaluate the model
# trainer.evaluate()

In [12]:
import torch
from transformers import DebertaV2Tokenizer, DebertaV2ForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset, load_metric

# Load the DeBERTa tokenizer and model
tokenizer = DebertaV2Tokenizer.from_pretrained('microsoft/deberta-v3-base')
model = DebertaV2ForSequenceClassification.from_pretrained('microsoft/deberta-v3-base', num_labels=3)  # Assuming binary classification

# Load your dataset (this example uses the GLUE MRPC dataset as an example)
#dataset = load_dataset("glue", "mrpc")

# # Tokenize the dataset
# def preprocess_function(examples):
#     return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding=True)


# Tokenize the dataset
train_input = tokenizer(fever_plus["train"]["premise"], fever_plus["train"]["hypothesis"], padding='max_length', truncation=True, max_length=128, return_tensors="pt")
val_input = tokenizer(fever_plus["validation"]["premise"], fever_plus["validation"]["hypothesis"], padding='max_length', truncation=True, max_length=128, return_tensors="pt")
test_input = tokenizer(fever_plus["test"]["premise"], fever_plus["test"]["hypothesis"], padding='max_length', truncation=True, max_length=128, return_tensors="pt")



# Split the dataset into train and test sets
train_dataset = train_input
eval_dataset = val_input #encoded_dataset['validation']

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# # Load the accuracy metric
# metric = load_metric("accuracy")

# # Define the compute_metrics function
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = torch.argmax(logits, dim=-1)
#     return metric.compute(predictions=predictions, references=labels)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset#,
    #compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()


c:\Users\damic\anaconda3\envs\nlp_env2\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overf

KeyError: 'Invalid key. Only three types of key are available: (1) string, (2) integers for backend Encoding, and (3) slices for data subsetting.'